# Análise de Biodiversidade da Mata Atlântica

## Objetivo
Analisar dados de espécies ameaçadas da Mata Atlântica usando dados do Sistema SALVE (ICMBio) para identificar padrões de biodiversidade e conservação.

## Fontes de Dados
- *ICMBio SALVE*: Sistema de Avaliação do Risco de Extinção da Biodiversidade
- *SiBBr*: Dados de ocorrência em Unidades de Conservação
- *Portal de Dados Abertos*: Datasets governamentais

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

c:\Users\Felipe\AppData\Local\Programs\Python\Python311\Lib\site-packages\seaborn\_statistics.py:32: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  from scipy.stats import gaussian_kde


In [2]:
# Configurar visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("Bibliotecas importadas com sucesso!")
print(f"Análise iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Bibliotecas importadas com sucesso!
Análise iniciada em: 2025-08-19 10:59:44


In [ ]:
# Função para testar conectividade com APIs do ICMBio
def testar_apis_icmbio():
    """Testa conectividade com diferentes APIs do ICMBio"""
    
    apis_teste = {
        'SALVE_BASE': 'https://salve.icmbio.gov.br/',
        'SIBBR': 'https://collectory.sibbr.gov.br/collectory/public/show/dr327'
    }
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cookie": "_ga_PFEFY1YDX4=GS2.1.s1755612408$o1$g0$t1755612408$j60$l0$h0; _ga=GA1.1.1874639878.1755612409",
    "Connection": "keep-alive"
    }


    resultados = {}
    
    for nome, url in apis_teste.items():
        try:
            response = requests.get(url, headers = headers, timeout=10)
            resultados[nome] = {
                'status': response.status_code,
                'acessivel': response.status_code == 200,
                'tamanho': len(response.content) if response.status_code == 200 else 0
            }
            print(f"{nome}: {response.status_code} - {'Acessível' if response.status_code == 200 else 'Erro'}")
        except Exception as e:
            resultados[nome] = {
                'status': 'Erro',
                'acessivel': False,
                'erro': str(e)
            }
            print(f"{nome}: Erro - {str(e)}")
    
    return resultados

In [10]:
# Testar APIs
print("Testando conectividade com APIs do ICMBio...")
status_apis = testar_apis_icmbio()

Testando conectividade com APIs do ICMBio...
SALVE_BASE: 403 - Erro
SIBBR: 200 - Acessível


In [34]:
# Planilha de espécies do Bioma Mata Atlântica foi baixada manualmente do site SALVE
# LICMBio, 2025. Sistema de Avaliação do Risco de Extinção da Biodiversidade – SALVE. Disponível em: https://salve.icmbio.gov.br/. Acesso em: 19 de Aug. de 2025.
df = pd.read_csv("../data/raw/Salve_Mata_Atlantica.csv")

ParserError: Error tokenizing data. C error: Expected 35 fields in line 240, saw 38


In [35]:
# como a leitura da planilha apresentou erro de parsing, vamos tentar descobrir o problema
# Testar diferentes parâmetros de leitura

try:
    df = pd.read_csv(
        "../data/raw/Salve_Mata_Atlantica.csv",
        encoding="utf-8",    # ou 'latin1' se necessário
        sep=",",             # ou ';' se necessário
        engine="python",
        on_bad_lines='skip'  # ignora linhas problemáticas
    )
    print(f"Arquivo lido com sucesso! Linhas lidas: {len(df)}")
except Exception as e:
    print("Erro ao ler o CSV:", e)

Arquivo lido com sucesso! Linhas lidas: 6206


In [36]:
# parece que algumas linhas estão mal formatadas, vamos contar quantas linhas foram puladas

with open("../data/raw/Salve_Mata_Atlantica.csv", encoding="utf-8") as f:
    total_linhas = sum(1 for _ in f)
print(f"Linhas no arquivo original: {total_linhas}")
print(f"Linhas lidas pelo pandas: {len(df)}")
print(f"Linhas puladas: {total_linhas - len(df)}")

Linhas no arquivo original: 6239
Linhas lidas pelo pandas: 6206
Linhas puladas: 33


In [ ]:
# Vamos descobrir quais são as colunas disponíveis no arquivo
# Com isso, podemos tentar ler a planilha original somente com as colunas que nos interessam
colunas = pd.read_csv(
    "../data/raw/Salve_Mata_Atlantica.csv",
    encoding="utf-8",
    sep=",",
    nrows=0
).columns.tolist()

print("Colunas disponíveis no arquivo:")
for c in colunas:
    print(c)

Colunas disponíveis no arquivo:
Reino
Filo
Classe
Ordem
Familia
Genero
Especie
Subespecie
Nome_Cientifico_Anterior
Autor
Nome_Comum
Grupo
Mes_Ano_Avaliacao
Categoria
Possivemente_Extinta
Criterio
Justificativa
Endemica_Brasil
Consta_em_Lista_Nacional_Oficial
Estado
Regiao
Bioma
Bacia_Hidrografica
Unidade_de_Conservacao_Federal
Unidade_de_Conservacao_Estadual
RPPN
Migratoria
Tendencia_Populacional
Ameaca
Uso
Acao_Conservacao
Plano_de_Acao
Listas_e_Convencoes
Altitude_min_max_
Barimetria_min_max_


In [37]:
# Agora que sabemos as colunas, vamos tentar ler apenas as que nos interessam
colunas_desejadas = [
    "Reino",
    "Filo",
    "Classe",
    "Ordem",
    "Familia",
    "Genero",
    "Especie",
    "Autor",
    "Nome_Comum",
    "Grupo",
    "Mes_Ano_Avaliacao",
    "Categoria",
    "Possivemente_Extinta",
    "Criterio",
    "Justificativa",
    "Endemica_Brasil",
    "Consta_em_Lista_Nacional_Oficial",
    "Estado",
    "Regiao",
    "Bioma",
    "Tendencia_Populacional",
    "Ameaca"
]

df = pd.read_csv(
    "../data/raw/Salve_Mata_Atlantica.csv",
    encoding="utf-8",
    sep=",",
    usecols=colunas_desejadas
)

print(df.head())

      Reino        Filo          Classe            Ordem         Familia  \
0  Animalia  Arthropoda         Insecta      Hymenoptera          Apidae   
1  Animalia    Chordata  Actinopterygii     Siluriformes    Loricariidae   
2  Animalia    Chordata  Actinopterygii    Characiformes      Bryconidae   
3  Animalia    Chordata        Reptilia         Squamata  Amphisbaenidae   
4  Animalia    Mollusca      Gastropoda  Caenogastropoda        Tateidae   

          Genero                 Especie                         Autor  \
0        Plebeia        Plebeia grapiuna            Melo & Costa, 2009   
1  Pareiorhaphis   Pareiorhaphis scutula  Pereira, Vieira & Reis, 2010   
2         Brycon           Brycon dulcis           Lima & Vieira, 2017   
3    Amphisbaena  Amphisbaena nigricauda                    Gans, 1966   
4   Potamolithus  Potamolithus karsticus    Simone & Moracchioli, 1994   

                          Nome_Comum                       Grupo  ...  \
0                        